In [2]:
from owl import *
from trueskill import Rating, rate, BETA, global_env
import itertools
import math

def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [12]:
matches = load_json('stats/matches.json')
teams = {}
players = {}
formations = {}
for match in matches:
    vs = []
    vs_formation = []
    all_players = []
    all_formations = []
    for i, team in enumerate(match['teams']):
        team_name = match['team_names'][i]
        team_players = []
        if team_name not in teams:
            teams[team_name] = {'players': {}, 'name': team_name, 'formations': {}}
        for player_name in team:
            if player_name not in players:
                players[player_name] = {'rate': Rating(), 'name': player_name}
            teams[team_name]['players'][player_name] = players[player_name]['rate']
            team_players.append(players[player_name]['rate'])
            all_players.append(player_name)
        formation_name = ','.join(team)
        if formation_name not in formations:
            formations[formation_name] = {'rate': Rating(), 'name': formation_name}
        teams[team_name]['formations'][formation_name] = formations[formation_name]['rate']
        vs.append(team_players)
        vs_formation.append([formations[formation_name]['rate']])
        all_formations.append(formation_name)
    ranks = match['ranks']
    ranks.reverse()
    new_ratings = rate(vs, ranks)
    new_ratings = list(new_ratings[0]) + list(new_ratings[1])
    for i, rating in enumerate(new_ratings):
        players[all_players[i]]['rate'] = rating
    new_ratings = rate(vs_formation, ranks)
    new_ratings = list(new_ratings[0]) + list(new_ratings[1])
    for i, rating in enumerate(new_ratings):
        formations[all_formations[i]]['rate'] = rating


In [15]:
teams.keys()

dict_keys(['SFS', 'VAL', 'SHD', 'GLA', 'DAL', 'SEO', 'LDN', 'FLA', 'PHI', 'HOU', 'BOS', 'NYE'])

In [24]:
sorted(teams['BOS']['players'].items(), key=lambda x:x[1].mu, reverse=True)

[('NotE', trueskill.Rating(mu=28.049, sigma=1.991)),
 ('DreamKazper', trueskill.Rating(mu=26.215, sigma=2.364)),
 ('Mistakes', trueskill.Rating(mu=25.302, sigma=2.884)),
 ('Kalios', trueskill.Rating(mu=25.282, sigma=3.910)),
 ('Kellex', trueskill.Rating(mu=25.142, sigma=1.914)),
 ('Neko', trueskill.Rating(mu=25.142, sigma=1.914)),
 ('Striker', trueskill.Rating(mu=25.031, sigma=1.920)),
 ('Gamsu', trueskill.Rating(mu=24.105, sigma=1.979))]

In [30]:
[(d['name'], d['rate']) for d in sorted(players.values(), key=lambda x: x['rate'].mu, reverse=True)]

[('super', trueskill.Rating(mu=30.984, sigma=2.917)),
 ('Geguri', trueskill.Rating(mu=29.351, sigma=3.192)),
 ('zunba', trueskill.Rating(mu=28.978, sigma=2.402)),
 ('cocco', trueskill.Rating(mu=28.912, sigma=3.347)),
 ('Hydration', trueskill.Rating(mu=28.819, sigma=2.391)),
 ('Fleta', trueskill.Rating(mu=28.593, sigma=2.233)),
 ('Custa', trueskill.Rating(mu=28.571, sigma=2.351)),
 ('LiNkzr', trueskill.Rating(mu=28.370, sigma=1.939)),
 ('Pine', trueskill.Rating(mu=28.112, sigma=3.174)),
 ('NotE', trueskill.Rating(mu=28.057, sigma=1.978)),
 ('tobi', trueskill.Rating(mu=28.001, sigma=2.206)),
 ('Moth', trueskill.Rating(mu=27.734, sigma=2.467)),
 ('Saebyeolbe', trueskill.Rating(mu=27.644, sigma=1.946)),
 ('Kariv', trueskill.Rating(mu=27.627, sigma=2.196)),
 ('neptuNo', trueskill.Rating(mu=27.454, sigma=1.864)),
 ('OGe', trueskill.Rating(mu=27.369, sigma=3.442)),
 ('Gesture', trueskill.Rating(mu=27.311, sigma=1.950)),
 ('Surefour', trueskill.Rating(mu=27.304, sigma=2.092)),
 ('ryujehong', t